In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression

In [2]:
# Membaca file Excel dengan banyak sheet
file_path = 'data/dataset_primer_2.xlsx' 
output_path = 'data/dataset_processed.xlsx'


In [3]:
def fill_missing_data_with_trend(df):
    target_columns = [col for col in df.columns if 'supply' not in col]

    for col in target_columns:
        # Pisahkan data yang sudah ada dan data yang hilang
        known_data = df[df[col].notna()]
        missing_data = df[df[col].isna()]

        if not known_data.empty and not missing_data.empty:
            # Menggunakan waktu sebagai fitur untuk memprediksi penurunan nilai
            X_train = known_data[['waktu']]  # Waktu sebagai fitur
            y_train = known_data[col]

            # Membangun model regresi linier
            model = LinearRegression()
            model.fit(X_train, y_train)

            # Prediksi untuk data yang hilang
            X_missing = missing_data[['waktu']]
            predicted_values = model.predict(X_missing)

            # Menurunkan nilai prediksi seiring waktu (dengan pengaturan skala penurunan)
            predicted_values = predicted_values - predicted_values.min()

            # Isi nilai yang hilang dengan prediksi yang telah disesuaikan
            df.loc[df[col].isna(), col] = predicted_values

    return df

In [4]:
# Fungsi untuk membaca dan memproses seluruh sheet dalam file Excel
def process_excel_file(file_path, output_path):
    xls = pd.ExcelFile(file_path)
    with pd.ExcelWriter(output_path) as writer:
        for sheet_name in xls.sheet_names:
            df = pd.read_excel(xls, sheet_name=sheet_name)
            df_filled = fill_missing_data_with_trend(df)
            df_filled.to_excel(writer, sheet_name=sheet_name, index=False)

In [5]:
process_excel_file(file_path, output_path)